# Advance Tools in OpenAI Agents SDK - 29 August 2025

### Configrations

In [ ]:
import os
from agents import set_default_openai_api, set_default_openai_client, set_tracing_disabled

from dotenv import find_dotenv, load_dotenv
from openai import AsyncOpenAI


load_dotenv(find_dotenv())
gemini_api_key = os.getenv("GEMINI_API_KEY")

if gemini_api_key is None:
    raise ValueError("Enviroment variable 'GEMINI_API_KEY' is not set.")


#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

set_default_openai_client(external_client)
set_tracing_disabled(disabled=True)
set_default_openai_api("chat_completions")


### tool_use_behavior

In [ ]:
from agents import Agent, Runner, function_tool

# Simple tools
@function_tool
def fetch_data():
    return { "name": "hamzah", "age": 24 }

@function_tool
def save_data():
    return "Saved data"

# Create agent
agent = Agent(
    name="Data Manager",
    tools=[fetch_data, save_data],
    model="gemini-2.0-flash",
    tool_use_behavior="stop_on_first_tool" # default "run_llm_again"
)

result = Runner.run_sync(
    starting_agent=agent,
    # input="hello, fetch user data for me"
    # input="fetch user data and save it in the database"
    input="save my name is the database: my name is hamzah"
)

print(result.final_output)


### StopAtTools

In [ ]:
from agents import Agent, Runner, function_tool, StopAtTools

# Simple tools
@function_tool
def fetch_data():
    return { "name": "hamzah", "age": 24 }

@function_tool
def save_data():
    return "Saved data"

# Create agent
agent = Agent(
    name="Data Manager",
    tools=[fetch_data, save_data],
    model="gemini-2.0-flash",
    tool_use_behavior=StopAtTools(stop_at_tool_names=["fetch_data"])
)

result = Runner.run_sync(
    starting_agent=agent,
    # input="hello, fetch user data for me"
    # input="fetch user data and save it in the database"
    input="save my name is the database: my name is hamzah"

)

print(result.final_output)


### name_override, description_override, is_enabled

In [ ]:
from agents import Agent, Runner, function_tool

# Simple tools
@function_tool
def fetch_data():
    return { "name": "hamzah", "age": 24 }

@function_tool(
    name_override="save_user_data", 
    description_override="this fn is saving user data",
    is_enabled=False,
)
def save_data():
    """this function is saving data"""
    return "Saved data"

# Create agent
agent = Agent(
    name="Data Manager",
    tools=[fetch_data, save_data],
    model="gemini-2.0-flash",
)

result = Runner.run_sync(
    starting_agent=agent,
    input="save my name is the database: my name is hamzah"
)

print(result.final_output)


### Dynamic is_enabled

In [ ]:
from agents import Agent, Runner, function_tool

# Simple tools
@function_tool
def fetch_data():
    return { "name": "hamzah", "age": 24 }

def check_is_admin():
    return False

@function_tool(
    name_override="save_user_data", 
    description_override="this fn is saving user data",
    is_enabled=check_is_admin(),
)
def save_data():
    """this function is saving data"""
    return "Saved data"

# Create agent
agent = Agent(
    name="Data Manager",
    tools=[fetch_data, save_data],
    model="gemini-2.0-flash",
    tool_use_behavior=StopAtTools(stop_at_tool_names=["fetch_data"])
)

result = Runner.run_sync(
    starting_agent=agent,
    input="save my name is the database: my name is hamzah"
)

print(result.final_output)


### failure_error_function

In [ ]:
from agents import Agent, Runner, function_tool

# Simple tools
@function_tool
def fetch_data():
    return { "name": "hamzah", "age": 24 }

def error_handler(context, error: Exception):
    # Send mail or msg on whatsapp
    print("email sent")
    return "Error handeled"

@function_tool(
    failure_error_function=error_handler
)
def save_data():
    """this function is saving data"""
    raise Exception("Error saving data")
    return "Saved data"

# Create agent
agent = Agent(
    name="Data Manager",
    tools=[fetch_data, save_data],
    model="gemini-2.0-flash",
)

result = Runner.run_sync(
    starting_agent=agent,
    input="save my name is the database: my name is hamzah"
)

print(result.final_output)
